In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
os.chdir("/home/sundari/MDM/Kaggle/Rossmann_Store_Sales")

In [2]:
df_train = pd.read_csv("train.csv",sep=",")
df_stores = pd.read_csv("store.csv",sep=",")

/home/sundari/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
#Merge the stores and train data
df_merged = pd.merge(df_train,df_stores,on='Store',how='outer')

In [4]:
df_merged.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270,9,2008,0,NaN,NaN,NaN
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270,9,2008,0,NaN,NaN,NaN
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270,9,2008,0,NaN,NaN,NaN
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270,9,2008,0,NaN,NaN,NaN
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270,9,2008,0,NaN,NaN,NaN


In [5]:
df_compitetion = df_merged.reindex(columns=['Store', 'Date', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear','Sales'])
df_compitetion.head()

,Store,Date,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Sales
0,1,2015-07-31,1270,9,2008,5263
1,1,2015-07-30,1270,9,2008,5020
2,1,2015-07-29,1270,9,2008,4782
3,1,2015-07-28,1270,9,2008,5011
4,1,2015-07-27,1270,9,2008,6102


In [6]:
df_compitetion['Date'] = pd.to_datetime(df_compitetion['Date'])
df_compitetion['data_year'] = df_compitetion['Date'].dt.year


In [7]:
#Replace CompetitionOpenSinceYear and CompetitionOpenSinceMonth NaN with zeros
df_compitetion['CompetitionOpenSinceYear'] = df_compitetion['CompetitionOpenSinceYear'].fillna(0)
df_compitetion['CompetitionOpenSinceMonth'] = df_compitetion['CompetitionOpenSinceMonth'].fillna(0)
df_compitetion['CompetitionDistance'] = df_compitetion['CompetitionDistance'].fillna(0)

In [8]:
df_compitetion['diff_year'] = (df_compitetion['data_year'] - df_compitetion['CompetitionOpenSinceYear']).astype(int)

In [9]:
df_compitetion['diff_year'] = df_compitetion['diff_year'] + (df_compitetion['CompetitionOpenSinceMonth'] /12)

In [10]:
df_compitetion['diff_year'] = np.where(df_compitetion['diff_year'] < 0 , 0,df_compitetion['diff_year'])
df_compitetion['diff_year'] = np.where(df_compitetion['diff_year'] > 1000,0,df_compitetion['diff_year'])

In [11]:
df_compitetion['CompitetionSinceYears'] = df_compitetion['diff_year']
df_compitetion = df_compitetion.drop('diff_year',axis=1)
df_compitetion = df_compitetion.drop('data_year',axis=1)
df_compitetion = df_compitetion.drop('CompetitionOpenSinceMonth',axis=1)
df_compitetion = df_compitetion.drop('CompetitionOpenSinceYear',axis=1)

In [12]:
df_corrsubset = df_compitetion.reindex(columns=['CompetitionDistance','CompitetionSinceYears','Sales'])

In [13]:
df_corrsubset.corr('pearson')
df_corrsubset.corr('spearman')

,CompetitionDistance,CompitetionSinceYears,Sales
CompetitionDistance,1.000000,-0.000196,-0.022257
CompitetionSinceYears,-0.000196,1.000000,-0.017673
Sales,-0.022257,-0.017673,1.000000


In [ ]:
#Fit a linear regression to see the relationship
from sklearn.ensemble import RandomForestRegressor
series_target = df_corrsubset['Sales']
df_corrsubset = df_corrsubset.drop('Sales',axis=1)
model_rf = RandomForestRegressor()
model_rf.fit(df_corrsubset,series_target)
predicted_rf = model_rf.predict(df_corrsubset)

In [ ]:
df_actVSpred = pd.DataFrame([series_target,predicted_rf])
df_actVSpred = pd.DataFrame.transpose(df_actVSpred)
df_actVSpred.columns = ['Actual','Predicted']

,Actual,Predicted
1017204,4771,4307.25697
1017205,4540,4307.25697
1017206,4297,4307.25697
1017207,3697,4307.25697
1017208,0,4307.25697
